In [1]:
### same as main.py but as a jpynb

import open3d as o3d 
import numpy as np
import trimesh

import datato3D as d3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Bounding Boxes

In [2]:

path = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\bettingen.txt"
bbox_product = d3d.read_lv95_do_rectangular_bbox(path)
#
print(bbox_product)

################### coordinate are copied from delivery note
shrink_factor = 0.03
bbox_product = d3d.shrink_bbox(bbox_product, shrink_factor)

POLYGON ((2616516 1268697, 2617397 1268697, 2617397 1269247, 2616516 1269247, 2616516 1268697))


In [3]:
bbox_working_region = d3d.buffer_polygon(bbox_product, 0.1)
#
print(bbox_working_region)

POLYGON ((2616529.215 1268427.636, 2616502.0040696044 1268428.9727872896, 2616475.0551953437 1268432.970275166, 2616448.6279096087 1268439.589965636, 2616422.976721607 1268448.7681074613, 2616398.3486663024 1268460.4163101192, 2616374.9809253304 1268474.4223950503, 2616353.09854281 1268490.6514760002, 2616332.9122580476 1268508.9472580475, 2616314.616476 1268529.13354281, 2616298.38739505 1268551.0159253306, 2616284.381310119 1268574.3836663028, 2616272.733107461 1268599.0117216073, 2616263.5549656358 1268624.6629096086, 2616256.935275166 1268651.0901953438, 2616252.9377872893 1268678.0390696046, 2616251.601 1268705.25, 2616251.601 1269238.75, 2616252.9377872893 1269265.9609303954, 2616256.935275166 1269292.9098046562, 2616263.5549656358 1269319.3370903914, 2616272.733107461 1269344.9882783927, 2616284.381310119 1269369.6163336972, 2616298.38739505 1269392.9840746694, 2616314.616476 1269414.86645719, 2616332.9122580476 1269435.0527419525, 2616353.09854281 1269453.3485239998, 2616374.98

# DEM


In [4]:
## if your DEM-file does not cover the area, try d3d.download_raster_files() and stitch_raster_files() to get a DEM that covers the area
## automatically download and stich raster files from the csv you get when you draw a polygon on https://www.swisstopo.admin.ch/de/hoehenmodell-swissalti3d

csv_path = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\swissaltiBettingen.csv"
output_directory = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\DEM"
raster_files = d3d.download_raster_files(csv_path, output_directory)

output_filename = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\DEM\merged_raster.tif"
d3d.stitch_raster_files(raster_files, output_filename)

In [5]:

dem_path = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\DEM\merged_raster.tif"
out_image, out_transform, src = d3d.read_raster_dem_cut_to_bbox(dem_path, bbox_working_region)
out_image = d3d.slice_out_image(out_image)
#
print(out_image.shape)

(545, 705)


In [6]:
# The parameters are (out_image, out_transform, resolution, max_triangle_size, max_triangle_height)
mesh, pcd = d3d.dem_to_mesh(out_image, out_transform, 0.1, 30, 8)

# Visualize the mesh and pointcloud
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)


In [7]:
mesh_cropped = d3d.cutting_mesh_with_bbox(mesh, bbox_product, 300, 600)
# Visualize the cut mesh and pointcloud
print(mesh, mesh_cropped)

o3d.visualization.draw_geometries([mesh_cropped], mesh_show_wireframe=True, mesh_show_back_face=True)


TriangleMesh with 62900 points and 125594 triangles. TriangleMesh with 16009 points and 31488 triangles.


In [8]:
# afterwork on the mesh
mesh_smooth = mesh_cropped.filter_smooth_taubin(number_of_iterations=3)
mesh_smooth.compute_triangle_normals()

TriangleMesh with 16009 points and 31488 triangles.

In [9]:
import os
output_directory = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\output"
output_file = os.path.join(output_directory, "output_mesh.ply")

try:
    os.makedirs(output_directory, exist_ok=True)  # Create the output directory if it doesn't exist
    o3d.io.write_triangle_mesh(output_file, mesh_smooth)
    print("PLY file written successfully.")
except Exception as e:
    print(f"Error writing PLY file: {str(e)}")


PLY file written successfully.


In [10]:
thickness = 100 #from lowest point of DEM to bottom of the output stl, standard value is calculated to be at sea level
# create a solid mesh from the surface mesh
solid_mesh = d3d.surface_to_volume(mesh_smooth, thickness)

# Visualize the updated mesh
o3d.visualization.draw_geometries([solid_mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

# Export the extruded mesh to a file
output_file = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\output\output_solid_mesh.stl"
o3d.io.write_triangle_mesh(output_file, solid_mesh) 



[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'positions'
[Open3D WARNING] Ignoring attribute 'colors' for TensorMap with primary key 'positions'
[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'indices'


True

# Buildings


In [11]:
### shrinking bbox so the buildings will fit proparly, but can result on slicing failures
shrink_factor = 0.001
bbox_product = d3d.shrink_bbox(bbox_product, shrink_factor)

In [12]:
### change to trimesh as it can cut through the middle of faces which open3d cannot
mesh_path = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\3DStadtmodell.obj"
mesh = trimesh.load_mesh(mesh_path)

mesh_cut = d3d.cutting_mesh_through_middle_of_faces(mesh, bbox_product)


In [13]:
# Define edge_vertices
edge_vertices = mesh_cut.outline().vertices
### Filtering the vertices so that only edge points are left for each side
coordinates = np.array(edge_vertices[:, :3])
combined_mesh = mesh_cut

combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'east')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'west')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'north')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'south')

### trying to get more slice reliability
trimesh.repair.fill_holes(combined_mesh)

import os
output_directory = r"C:\Users\kuhnt\OneDrive - FHNW\G6\General - P-BTh-25-FS2024-Geometrieverarbeitung_M365\40_Software_entwicklung\Anwendung_Stadt\Bettingen\output"
output_file = os.path.join(output_directory, "output_combined_mesh.stl")

try:
    os.makedirs(output_directory, exist_ok=True)  # Create the output directory if it doesn't exist
    combined_mesh.export(output_file)
    print("STL file written successfully.")
except Exception as e:
    print(f"Error writing STL file: {str(e)}")

combined_mesh.show()

STL file written successfully.


# testing
